# Model of a lenslet

This notebook shows how a lenslet is modeled and how to estimate its parameters

## Packages installation

* `OptimPackNextGen` is taken on FerreolS repository until Eric accepts the changes for automatic differentiation in his repository,
* `Zygote` is an automatic differentiation package,
* `TwoDimensional` is an Eric's package that defines the bounding boxes and operations on it,
* `StatsBase` contains `mean`,
* `Plots` is for plotting. It is set with `plotly` backend and the packages `PlotlyBase` and `PlotlyJS` must be installed.

In [1]:
using Pkg;
Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")


    Updating git-repo `https://github.com/FerreolS/OptimPackNextGen.jl`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
include("../src/SphereIFSCalib.jl")
using TwoDimensional, Zygote,StatsBase, Plots, OptimPackNextGen
plotly()

Plots.PlotlyBackend()

## Parameters settings

In [3]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3,λ4]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument 


In [4]:
# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
a0= 1. .- 0.2 .* randn(Float64,length(λlaser));
fwhm0= 5. .-  2. .* rand(Float64,length(λlaser));
laser =  LaserModel(λlaser,a0,fwhm0);
lmod = LensletModel(λ0,laser.nλ-1,bbox);

C0 = zeros(2,laser.nλ)
C0[1:2,1:2] = [[ 6.2 10]; [25 80]]
UpdateDispModel(lmod.dmodel,C0)
@show lmod.dmodel.(λlaser)
limage = LensletLaserImage(lmod,laser)
wxy0 = lmod.dmodel.(wavelengthrange)
(wx0,wy0) = collect.( zip(wxy0...))
heatmap(limage)
plot!(wy0,wx0; label="dispersion law")


lmod.dmodel.(λlaser) = [(8.243979943212711, 41.35183954570169), (7.148589379568659, 32.58871503654927), (5.653110211643409, 20.624881693147273), (3.754320465575223, 5.434563724601784)]


<!DOCTYPE html>
 
 
 Plots.jl

In [5]:

data= limage .+ 0.05 .*randn(Float64,size(limage)) ;
heatmap(data; title="data")

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end
@show cost = likelihood(a0,fwhm0,C0);
@show ∇cost = gradient(likelihood,a0,fwhm0,C0);


In [ ]:
@show  cost = likelihood(a0,fwhm0.+ 2.,C0)
@time  ∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,C0);


cost = likelihood(a0, fwhm0 .+ 2.0, C0) = 19.30041682567078
  0.001745 seconds (10.91 k allocations: 556.375 KiB)


([16.01158705070926, 18.68878683758076, 15.881575258957596, 15.315220733587974], [4.410102413142761, 5.185692249114614, 4.8923340193472065, 3.2450954107999794], [-1.7956623543767942 -0.764928550267737 -0.013438005317553972 -0.034707848771226624; 0.06408003743805879 -0.23478165017013336 -0.037974822291517905 -0.01322053855326481])

In [ ]:

function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:(laser.nλ)],x[(laser.nλ+1):(2*laser.nλ)],reshape(x[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end


likelihood (generic function with 2 methods)

In [ ]:
x0 = vcat([a0[:],fwhm0[:],C0[:] ]...)
@show cost = likelihood(x0)
@time ∇cost = gradient(likelihood,x0);


cost = likelihood(x0) = 4.960410336257174
  0.001871 seconds (11.04 k allocations: 566.219 KiB)


In [ ]:

ainit = a0  .+ (rand(Float64,laser.nλ) .- 0.5);
fwhminit = fwhm0  .+ (rand(Float64,laser.nλ) .- 0.5);
cinit = zeros(2,laser.nλ)
cinit[1:2,1:2] = [[ 6.2 0]; [25 60]]

UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);
wxyinit = lmod.dmodel.(wavelengthrange)
(wxinit,wyinit) = collect.( zip(wxyinit...))

xinit = vcat([ainit[:],fwhminit[:],cinit[:]]...)



@time  ∇cost = gradient(likelihood,xinit )

@time xopt = vmlmb(likelihood, xinit; verb=50, ftol=(0.0,0),gtol = (0.0,0));

  0.001394 seconds (11.04 k allocations: 566.219 KiB)
# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    4.7806142547124630E+01  2.82E+01  0.00E+00
    50     53      0    1.0363312950094862E+01  7.97E+00  2.73E-01
   100    109      0    1.3464509861898071E+00  8.29E-02  1.00E+00
   150    162      0    1.2626878897844398E+00  6.99E-03  1.00E+00
   200    213      0    1.2622685882336004E+00  9.14E-03  1.00E+00
   250    265      0    1.2610596853596061E+00  8.77E-03  1.00E+00
   300    321      0    1.2561276446440406E+00  8.69E-03  1.00E+00
   350    374      0    1.2558535737437446E+00  1.03E-04  1.00E+00
   400    430      0    1.2558532680492585E+00  6.29E-04  3.73E-01
   450    485      0    1.2558529337036348E+00  7.78E-04  1.00E+00
   500    536      0    1.2558523203161449E+00  4.36E-05  1.00E+00
   550    592      0    1.2558520174948020E+00  1.10E-04  1.00E+00
   600    

In [ ]:
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
@show(ainit,aopt,a0);

@show(fwhminit,fwhmopt,fwhm0);

@show(cinit,copt,C0);

heatmap( data .- LensletLaserImage(lmod,laser))


ainit = [1.2912207652820555, 1.235661901459077, 1.3053261999822223, 0.7753021626639534]
aopt = [1.0081500713524012, 1.0677116301982943, 1.0624135776058177, 1.025043157676087]
a0 = [1.0311131795566892, 1.061861704874012, 1.054831748965497, 1.0328905420151948]
fwhminit = [4.089395131879767, 4.735445384999915, 3.4063185070045683, 5.1973609185425085]
fwhmopt = [3.9092240787800434, 4.7947269316323045, 3.553672992247917, 5.058107501063979]
fwhm0 = [3.9206558722751694, 4.77395400642536, 3.583085467238525, 4.966931906027826]
cinit = [6.2 0.0 0.0 0.0; 25.0 60.0 0.0 0.0]
copt = [6.237417747269071 9.397931393466413 -0.6026152551082388 14.534421661503844; 24.985657891964156 80.4670414833642 0.9295977874729339 -5.797285228444003]
C0 = [6.2 10.0 0.0 0.0; 25.0 80.0 0.0 0.0]


<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
wavelength = LinRange(850e-9,1350e-9,50);
wxy = lmod.dmodel.(wavelengthrange)
(wx,wy) = collect.( zip(wxy...))
heatmap(data)
plot!(wy0,wx0;label="truth")
plot!(wy,wx; label="estimate")
plot!(wyinit,wxinit; label="init")

<!DOCTYPE html>
 
 
 Plots.jl